In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
import torch
from transformers import TrainingArguments, Trainer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import plot_confusion_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification  
import pandas as pd
import re

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
# model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")

In [ ]:
# classes = ["NOT_oFF", "OFF"]
df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/offLabel.xlsx')

#Label Encoding to convert labels to integer values
y_test = df_test_labels.labels
LE = LabelEncoder()
y_test=LE.fit_transform(y_test)
print(y_test[:20])
# pred1 = []
# df_test_labels.head
# df_test.head

[1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]


In [ ]:
!git clone https://github.com/aub-mind/arabert
!pip install -q farasapy
!pip install -q pyarabic
from arabert.preprocess import ArabertPreprocessor

arabert_prep = ArabertPreprocessor(model_name='bert-base-arabert')
# arabert_prepv2 = ArabertPreprocessor(model_name='bert-base-arabertv2')


fatal: destination path 'arabert' already exists and is not an empty directory.
     |████████████████████████████████| 112kB 31.7MB/s 


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:20<00:00, 11.1MiB/s]

[2021-06-06 12:58:11,149 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


#Preprocess 

In [ ]:
text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>:@,\\-_”“٪ًَ]"
def preprocess(text):
  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", str(text))
  # text = text.replace('وو', 'و')
  # text = text.replace('يي', 'ي')
  # text = text.replace('اا', 'ا')
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  #remove longation
  text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
  text = re.sub("[إأآا]", "ا", str(text))
  text = re.sub(r"([?!¿])", r" \1 ", str(text))
  text = re.sub("ى", "ي", str(text))
  # text = re.sub("ؤ", "ء", text)
  # text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", str(text))
  # for i in range(0, len(search)):
  #       text = text.replace(search[i], replace[i])
  
   #trim    
  # text = text.strip()

  tokens = []
  for token in text.split():
    # if token not in stop_words:
      tokens.append(token)
  return " ".join(tokens)

#Build Model 

In [ ]:
from transformers import pipeline
Model='vicgalle/xlm-roberta-large-xnli-anli'

classifier = pipeline("zero-shot-classification", model=Model,tokenizer=Model)

In [ ]:
df_test.tweets = df_test.tweets.apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
y_pred_new=[]
for tweet in df_test.tweets:
    pred_zero=classifier(tweet, ["toxic", "not_toxic"])
    # pred_zero=list(pred_zero.items())
    # case = {pred_zero['sequence'] : tweet[0], pred_zero['scores']: tweet[1], pred_zero['labels']:tweet[2] }
    # case = {'key1': pred_zero['scores'][0], 'key2': pred_zero['scores'][1] }
    if(pred_zero["scores"][0]> pred_zero["scores"][1]):
      y_pred_new.append(pred_zero["labels"][0])
    else:
      y_pred_new.append(pred_zero["labels"][1])

y_pred_new[:20]

['toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'not_toxic',
 'toxic',
 'not_toxic',
 'toxic',
 'not_toxic']

In [ ]:
#Label Encoding to convert labels to integer values
y_pred=LE.fit_transform(y_pred_new)
print(y_pred[:20])

[1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0]


In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))

print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.809

Confusion matrix
 [[1475  123]
 [ 259  143]]

Classification report
               precision    recall  f1-score   support

           0       0.85      0.92      0.89      1598
           1       0.54      0.36      0.43       402

    accuracy                           0.81      2000
   macro avg       0.69      0.64      0.66      2000
weighted avg       0.79      0.81      0.79      2000



In [ ]:
#  pred_zero=classifier('USER @USER  المسلمين والمسلمات شفاء لا يغادر سقما واصبغ عليهم عافيتك يا شافي يا معافى يا ارحم الراحمين.', ["hate", "not_hate"], device=0)
pred_zero=classifier(' #الدورة_الشهرية يا عديمين الرجولة يا الذكور يا اشباة الرجال يا عبيد الكورة الله يحرقكم', ["hate", "not_hate"], device=0)
    # pred_zero=list(pred_zero.items())
    # case = {pred_zero['sequence'] : tweet[0], pred_zero['scores']: tweet[1], pred_zero['labels']:tweet[2] }
    # case = {'key1': pred_zero['scores'][0], 'key2': pred_zero['scores'][1] }
SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
y_pred = np.argmax(SCORES)
print(SCORES)
pred_zero["labels"]

[0.988742470741272, 0.011257510632276535]


['hate', 'not_hate']

# Experments

In [ ]:
# paraphrase = tokenizer(sequence_0, return_tensors="pt")
# 'not_paraphrase = tokenizer( sequence_1, return_tensors="pt")

In [ ]:
# # not_paraphrase_classification_logits = model(**not_paraphrase).logits
# 'paraphrase_classification_logits = model(**paraphrase).logits

In [ ]:
# # paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
# 'not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]


In [ ]:
# def comparelabel(label, predect):
#   if(label == predect):
#     return "success"
#   return "failed"

# for i in range(len(classes)):
# pred.append(int(paraphrase_results[0] < 0.5))
# pred.append(int(not_paraphrase_results[0] < 0.5))
# if(paraphrase_results[0] > 0.5):
#   print(f"{classes[0]}: {int(round(paraphrase_results[0] * 100))}% - label: {classes[label[0]]} - predict: {comparelabel(label[0],int(paraphrase_results[0] < 0.5))}")
# else:
#   print(f"{classes[1]}: {int(round(paraphrase_results[1] * 100))}% - label: {classes[label[0]]} - predict: {comparelabel(label[0],int(paraphrase_results[0] < 0.5))}")
# # print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}% - label: {classes[label[k]]} - predict: {comparelabel()}")

In [ ]:
# for i in range(len(classes)):
#     print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

In [ ]:
# df_test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
# df_test_labels = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/testLabels.xlsx')

In [ ]:
# df_test_new=pd.concat([df_test, df_test_labels], axis=1)
# df_test_new=df_test_new.drop(columns=['id'])

# df_test_new

In [ ]:
# grouped = df_test_new.groupby(df_test_new.label)
# y_test_HS = grouped.get_group("HS")[:50]
# y_test_Not_HS = grouped.get_group("NOT_HS")[:50]
# # y_test_HS=y_test_HS.label.values
# # y_test_Not_HS=y_test_Not_HS.label.values

# # print(y_test_HS.head())
# # print(y_test_Not_HS.head())
# # print('==========================================================================')
# print(' Hate Speech label shape | ',y_test_HS.shape)
# print(' Not Hate Speech label shape | ',y_test_Not_HS.shape)

# %load_ext google.colab.data_table
# y_test_HS


In [ ]:
# %load_ext google.colab.data_table
# y_test_Not_HS

In [ ]:
# # toknize_test_HS = tokenizer(y_test_HS.tweets.to_list(),   max_length=120,padding=True, truncation=True,return_tensors="pt")
# toknize_test_Not_HS = tokenizer(y_test_Not_HS.tweets.to_list(),   max_length=120,padding=True, truncation=True,return_tensors="pt")
# hs_classification_logits = model(**toknize_test_HS).logits
# not_hs_classification_logits = model(**toknize_test_Not_HS).logits
# import torch

# hs_results = torch.softmax(hs_classification_logits, dim=1)
# not_hs_results = torch.softmax(not_hs_classification_logits, dim=1)
# not_hs_results.tolist()

In [ ]:
# y_pred = np.argmax(not_hs_results, axis=1)

In [ ]:
# # Should be paraphrase
# classes = ["HS", "NOT-HS"]
# for i in range(len(classes)):
#     print(hs_results[i])
#     # print(f"{classes[i]}: {int(round(hs_results[i] * 100))}%")

In [ ]:
# for i in range(len(classes)):
#     print(not_hs_results[i])  
#     # print(f"{classes[i]}: {int( torch.round(not_hs_results[i] * 100))}%")

In [ ]:
# test_encodings=tokenizer(list(x_test),truncation=True,padding=True)
# test_encodings

In [ ]:
# # Create torch dataset
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels=None):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels:
#             item["labels"] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])

In [ ]:
# # Load test data
# # test_data = pd.read_csv("test.csv")
# X_test = list(df_test_new["tweets"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

In [ ]:
# test_trainer=Trainer(model)

In [ ]:
# raw_pred = test_trainer.predict(test_dataset)
# raw_pred, _, _ = test_trainer.predict(test_dataset)

In [ ]:
# y_pred = np.argmax(raw_pred, axis=1)
# y_pred[:20]

In [ ]:
#Label Encoding to convert labels to integer values
# y_test=df_test_new.label
# LE = LabelEncoder()
# y_test=LE.fit_transform(y_test)
# y_test[:20]

In [ ]:
# accuracy_score(y_test, y_pred)

In [ ]:
# accuracy = accuracy_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print("accuracy", accuracy_score(y_test, y_pred))
# print("precision", precision)
# print( "recall", recall)
# print( "f1",f1)
# print('--------')
# print(classification_report(y_test,y_pred))
# print(confusion_matrix(y_test,y_pred))
